# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount pearl,47.5166,-52.7813,15.19,93,40,5.66,Mount Pearl,1693373996
1,1,laguna,38.4210,-121.4238,25.35,54,0,1.54,Laguna,1693374274
2,2,port mathurin,-19.6833,63.4167,22.64,73,16,5.60,Port Mathurin,1693374002
3,3,flying fish cove,-10.4217,105.6791,25.82,78,100,6.17,Flying Fish Cove,1693374020
4,4,kariapatti,9.6751,78.0999,32.29,58,40,2.57,Kariapatti,1693374354


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    scale = 0.5,
    tiles = "OSM"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Cloudiness'] < 5) & (city_data_df['Humidity'] < 80)
]
# Drop any rows with null values
cleaned_df = filtered_df.dropna()

# Display sample data
cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,laguna,38.4210,-121.4238,25.35,54,0,1.54,Laguna,1693374274
24,24,port alfred,-33.5906,26.8910,21.72,32,0,4.18,Port Alfred,1693374013
44,44,shevchenko,48.5000,35.0000,24.55,23,0,3.74,Shevchenko,1693374362
64,64,bolotnoye,55.6717,84.3981,22.35,46,1,4.55,Bolotnoye,1693374368
110,110,glubokoye,50.1456,82.3114,23.86,50,0,3.00,Glūbokoe,1693374379
114,114,shahrak,34.1074,64.3052,23.86,11,0,0.86,Shahrak,1693374155
115,115,amga,60.8953,131.9608,26.09,41,0,1.85,Amga,1693374381
127,127,yanchep,-31.5500,115.6833,22.75,51,0,3.40,Yanchep,1693374141
134,134,tamanrasset,22.7850,5.5228,21.95,46,0,2.06,Tamanghasset,1693374385
188,188,nova xavantina,-14.6667,-52.3500,25.54,60,3,1.33,Nova Xavantina,1693374397


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df.copy()
selected_columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = hotel_df[selected_columns]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,laguna,Laguna,38.4210,-121.4238,54,
24,port alfred,Port Alfred,-33.5906,26.8910,32,
44,shevchenko,Shevchenko,48.5000,35.0000,23,
64,bolotnoye,Bolotnoye,55.6717,84.3981,46,
110,glubokoye,Glūbokoe,50.1456,82.3114,50,
114,shahrak,Shahrak,34.1074,64.3052,11,
115,amga,Amga,60.8953,131.9608,41,
127,yanchep,Yanchep,-31.5500,115.6833,51,
134,tamanrasset,Tamanghasset,22.7850,5.5228,46,
188,nova xavantina,Nova Xavantina,-14.6667,-52.3500,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    scale = 0.5,
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)